In [1]:
import torch
import pandas as pd
import torch.nn as nn

test_df = pd.read_csv('./data/test_df.csv', index_col=0)
means = pd.read_csv('./data/means.csv', index_col = 0)
stds = pd.read_csv('./data/stds.csv', index_col = 0)

y_cols = ["Målt konsistens [mm]", "Målt densitet [kg/m3]", "Målt luftinnhold [%]", "Sylinder 28 døgn"]

X_test_tensor = torch.tensor(test_df.drop(columns=y_cols).values, dtype=torch.float32)
y_test_tensor = torch.tensor(test_df[y_cols].values, dtype=torch.float32)

print("test data loaded successfully")


class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the path to the saved model
model_path = './models/simple_nn_model.pth'

# Load the model
model = SimpleNN(X_test_tensor.shape[1], y_test_tensor.shape[1])
model.load_state_dict(torch.load(model_path))
model.eval() 

print("Model loaded successfully.")


y_stds_tensor = torch.tensor(stds.loc[y_cols].to_numpy(), dtype=torch.float32).view(-1, 1)
y_means_tensor = torch.tensor(means.loc[y_cols].to_numpy(), dtype=torch.float32).view(-1, 1)
X_stds_tensor = torch.tensor(stds.drop(index=y_cols).to_numpy(), dtype=torch.float32).view(-1, 1)
X_means_tensor = torch.tensor(means.drop(index=y_cols).to_numpy(), dtype=torch.float32).view(-1, 1)

def denormalizeInputs(inputs):
    return inputs.view(-1, 1) * X_stds_tensor + X_means_tensor

def denormalizeOutputs(outputs):
    return torch.exp(outputs.view(-1, 1) * y_stds_tensor + y_means_tensor)

    

test data loaded successfully
Model loaded successfully.


In [2]:
material_costs_df = pd.read_csv("./data/material_costs_estimates.csv", index_col=0).drop(columns=["Category"])
material_costs_tensor = torch.tensor(material_costs_df.to_numpy(), dtype=torch.float32)

def materials_cost(inputs):
    return (inputs * material_costs_tensor).sum()


In [3]:
def objective_function(inputs, doPrint=False, min_strength = 60):
    
    outputs = denormalizeOutputs(model(inputs))
    strength = outputs[3, -1]
    cost = materials_cost(denormalizeInputs(inputs))

    if (doPrint):
        print("STRENGTH:", strength.item(), "COST:", cost.item())

    objective = torch.where(strength < min_strength, strength, -(cost ** 2) / 1.7e9                            )
    return objective

In [ ]:
# DEBUGGING

# inputs = torch.zeros(1, X_test_tensor.shape[1], requires_grad=True)
# objective_function(inputs, doPrint=True)

STRENGTH: 55.236175537109375 COST: 322037.75


tensor(55.2362, grad_fn=<WhereBackward0>)

In [5]:

SAMPLES = 20
ITERATIONS = 1500
MIN_STRENGTH = 41

for i in range(SAMPLES):

    inputs = torch.normal(mean=0, std=1, size=(1, X_test_tensor.shape[1]), requires_grad=True)

    best_inputs = inputs
    lowest_cost = materials_cost(denormalizeInputs(inputs))


    for j in range(ITERATIONS):

        outputs = model(inputs)

        # Compute gradients without backward()
        doPrint = j == ITERATIONS -1
        # doPrint = False
        grads = torch.autograd.grad(objective_function(inputs, doPrint=doPrint, min_strength=MIN_STRENGTH), inputs, grad_outputs=torch.ones_like(objective_function(inputs)), retain_graph=True)[0]

        inputs = torch.clamp(inputs + grads * 0.01, min=-(X_means_tensor / (X_stds_tensor - 1e-8)).view(1, -1))

        # print("OUTPUT:", denormalizeOutputs(model(inputs)))
    
    cost = materials_cost(denormalizeInputs(inputs))
    if (cost < lowest_cost):
        lowest_cost = cost
        print("NEW BEST:")
        objective_function(inputs, doPrint=True, min_strength=MIN_STRENGTH)
        
    # print(denormalizeInputs(inputs))
    # print(denormalizeOutputs(outputs))



STRENGTH: 41.03995895385742 COST: 20426.001953125
NEW BEST:
STRENGTH: 41.0305290222168 COST: 20414.078125
STRENGTH: 41.92363739013672 COST: 25256.78515625
NEW BEST:
STRENGTH: 41.778480529785156 COST: 23660.072265625
STRENGTH: 41.21866226196289 COST: 21029.21484375
NEW BEST:
STRENGTH: 41.20625305175781 COST: 21013.267578125
STRENGTH: 41.091064453125 COST: 18075.20703125
NEW BEST:
STRENGTH: 41.08549499511719 COST: 18065.3984375
STRENGTH: 41.328590393066406 COST: 21075.72265625
NEW BEST:
STRENGTH: 41.30047607421875 COST: 20977.9921875
STRENGTH: 41.25511932373047 COST: 21886.203125
NEW BEST:
STRENGTH: 41.24259948730469 COST: 21869.205078125
STRENGTH: 41.06464385986328 COST: 15052.60546875
NEW BEST:
STRENGTH: 41.05857467651367 COST: 15045.68359375
STRENGTH: 41.56645584106445 COST: 19957.92578125
NEW BEST:
STRENGTH: 41.528961181640625 COST: 19819.994140625
STRENGTH: 41.29393768310547 COST: 20217.103515625
NEW BEST:
STRENGTH: 41.28385925292969 COST: 20201.8125
STRENGTH: 41.86660385131836 COST